### Required Libraries

In [ ]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from transformers import BertTokenizer
import torch
nltk.download('stopwords')

### Loading Dataset

In [ ]:
df = pd.read_csv('projectdataset.csv')

### Explorartory Data Analysis

#### Class Balance of Dataset

In [ ]:
df["Own_Rating"].value_counts()

#### No. of categories and their count

In [ ]:
df["Category"].value_counts()

#### Column names of Dataset

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
def pre_process(df):

    df.dropna(subset=['Review_text'] , inplace = True)
    df['Review_text'] = df['Review_text'].apply(lambda x: x.lower())

    def remove_stopwords_and_punctuation(text):
        stop_words = set(stopwords.words('english'))

        text = ''.join([char for char in text if char not in string.punctuation])
        words = text.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return ' '.join(filtered_words)


    df['Review_text'] = df['Review_text'].apply(remove_stopwords_and_punctuation)



pre_process(df)

In [ ]:
df['Review_text']

In [ ]:
df.drop(columns=['Unique_ID' , 'Category' , 'Review_Header' , 'Rating'] , inplace=True)

In [ ]:
df

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


def encode_texts(texts, tokenizer, max_len=128):
    return tokenizer.batch_encode_plus(
        texts.tolist(),
        max_length=max_len,
        padding='max_length',  # Pad shorter sequences
        truncation=True,  # Truncate longer sequences
        return_tensors='pt'  # Return as PyTorch tensors
    )

def encode_labels(labels):
    # Create a mapping for your sentiment ratings
    sentiment_dict = {'Positive': 1, 'Neutral': 0, 'Negative': 2}
    return torch.tensor([sentiment_dict[label] for label in labels.tolist()])


encoded_inputs = encode_texts(df['Review_text'], tokenizer)
encoded_labels = encode_labels(df['Own_Rating'])


class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx]
        }


input_ids = encoded_inputs['input_ids']
attention_masks = encoded_inputs['attention_mask']
labels = encoded_labels

dataset = SentimentDataset(input_ids, attention_masks, labels)


from torch.utils.data import DataLoader

batch_size = 16  # Adjust based on your hardware
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)




In [ ]:
import pandas as pd

# Extract input IDs, attention masks, and labels from the dataset
input_ids_list = [dataset[i]['input_ids'].tolist() for i in range(len(dataset))]
attention_masks_list = [dataset[i]['attention_mask'].tolist() for i in range(len(dataset))]
labels_list = [dataset[i]['labels'].item() for i in range(len(dataset))]

# Convert to a DataFrame
df_bert = pd.DataFrame({
    'input_ids': input_ids_list,
    'attention_mask': attention_masks_list,
    'labels': labels_list
})


In [ ]:
df_bert

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertForSequenceClassification, AdamW, BertTokenizer
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Load your preprocessed dataset
dataframe = df_bert

# Convert the columns containing strings to lists
dataframe['input_ids'] = dataframe['input_ids'].apply(lambda x: eval(x) if isinstance(x, str) else x)
dataframe['attention_mask'] = dataframe['attention_mask'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Custom dataset class for DataLoader
class SentimentDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Train-test split
train_dataframe, val_dataframe = train_test_split(dataframe, test_size=0.2, random_state=42)

# Create datasets
train_dataset = SentimentDataset(train_dataframe['input_ids'].tolist(), train_dataframe['attention_mask'].tolist(), train_dataframe['labels'].tolist())
val_dataset = SentimentDataset(val_dataframe['input_ids'].tolist(), val_dataframe['attention_mask'].tolist(), val_dataframe['labels'].tolist())

# DataLoaders
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Initialize BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Use GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)

# Total training steps
epochs = 50
total_steps = len(train_dataloader) * epochs

# Scheduler for learning rate decay
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-tuning loop
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    # Training phase
    model.train()
    total_train_loss = 0
    correct_train_preds = 0

    for batch_idx, batch in enumerate(train_dataloader):
        optimizer.zero_grad()

        # Move batch to the device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()
        scheduler.step()

        # Predictions
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        correct_train_preds += (preds == labels).sum().item()

        # Print batch-level training stats
        if (batch_idx + 1) % 10 == 0:  # Print every 10 batches
            avg_batch_train_loss = total_train_loss / (batch_idx + 1)
            train_accuracy = correct_train_preds / ((batch_idx + 1) * batch_size)
            print(f"Batch {batch_idx+1}/{len(train_dataloader)} - Training loss: {avg_batch_train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")

    avg_train_loss = total_train_loss / len(train_dataloader)
    train_accuracy = correct_train_preds / len(train_dataset)
    print(f"Training loss: {avg_train_loss:.4f}")
    print(f"Training accuracy: {train_accuracy:.4f}")

    # Validation phase
    model.eval()
    total_val_loss = 0
    correct_val_preds = 0

    with torch.no_grad():
        for batch_idx, batch in enumerate(val_dataloader):
            # Move batch to the device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

            # Predictions
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct_val_preds += (preds == labels).sum().item()

            # Print batch-level validation stats
            if (batch_idx + 1) % 10 == 0:  # Print every 10 batches
                avg_batch_val_loss = total_val_loss / (batch_idx + 1)
                val_accuracy = correct_val_preds / ((batch_idx + 1) * batch_size)
                print(f"Batch {batch_idx+1}/{len(val_dataloader)} - Validation loss: {avg_batch_val_loss:.4f}, Validation accuracy: {val_accuracy:.4f}")

    avg_val_loss = total_val_loss / len(val_dataloader)
    val_accuracy = correct_val_preds / len(val_dataset)
    print(f"Validation loss: {avg_val_loss:.4f}")
    print(f"Validation accuracy: {val_accuracy:.4f}")

# Save the fine-tuned model
model.save_pretrained('fine_tuned_bert')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer.save_pretrained('fine_tuned_bert')
